In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
warnings.filterwarnings('ignore',message='profile')
warnings.filterwarnings('ignore',message='_astropy_init')
warnings.filterwarnings('ignore',message='ConfigurationDefaultMissingWarning ')

import os
import sys

In [6]:
from astroquery.vizier import Vizier
import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

#import pyvo as vo

In [4]:
Vizier.ucd  = 'phot.flux.density;em.radio.*'

In [22]:
def create_lsm(ra, dec, radius):
    """
    ra     : float in degrees with the right ascension of the target direction
    dec    : float in degrees with the declination of the target direction
    radius : float in degrees with the radius around the target to search for sources
    """
    
    Vizier.ucd  = 'phot.flux.density;em.radio.*'
    radius = str(radius) + 'd'
    
    if -30 >= dec >= -90:
        cat = 'The SUMSS Catalog, Version 2.1r (Feb-16-2012)'
        catalog_list = Vizier.find_catalogs('VIII/81B/sumss212') 
    else:
        cat = 'The NVSS Catalog'
        catalog_list = Vizier.find_catalogs('VIII/65/nvss') 
        
    result = Vizier.query_region(coord.SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='icrs'), 
                                 radius=radius, catalog=catalog_list.keys())
    
    
    for i in range(len(result)):
        if cat==result[i].meta['description']:
            print(i, result[i].meta['description'])
            chosen_cat = i
            
    MyCat = result[result.keys()[chosen_cat]]
    
    return MyCat

In [23]:
lsm = create_lsm(0, -31, 10)

(34, u'The SUMSS Catalog, Version 2.1r (Feb-16-2012)')


In [21]:
lsm

RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Sp,e_Sp,St,e_St,MajAxis,MinAxis,PA,Mosaic,Nm
"""h:m:s""",arcs,"""d:m:s""",arcs,mJy,mJy,mJy,mJy,arcs,arcs,deg,,
str11,float32,str12,float32,float32,float32,float64,float32,float32,float32,float32,str8,uint8
00 00 00.00,3.6,-31 09 53.10,4.7,13.1,1.4,13.2,1.4,84.9,58.4,162.3,J0000M32,1
00 00 02.25,2.3,-37 08 00.00,3.4,13.2,1.2,13.2,1.2,76.6,45.3,177.2,J0000M36,1
00 00 03.06,5.9,-33 00 34.90,6.9,12.6,1.6,24.6,3.0,121.2,73.2,144.7,J0000M32,1
00 00 06.15,4.6,-29 55 10.70,7.7,11.8,2.1,13.5,2.4,93.2,51.3,169.1,J0000M32,1
00 00 06.42,4.0,-34 19 08.20,5.8,10.4,1.6,10.4,1.6,76.6,49.5,11.6,J0000M36,1
00 00 06.67,3.0,-38 07 23.90,3.2,17.0,1.3,18.9,1.5,70.0,61.7,39.2,J0000M40,1
00 00 08.22,3.1,-37 38 19.60,3.3,18.1,1.5,19.1,1.5,76.6,59.9,41.6,J0000M36,1
00 00 09.97,1.6,-31 33 30.90,2.1,44.7,1.9,44.8,1.9,84.9,45.1,0.9,J0000M32,1


In [19]:
# LSM should be of the following format:
# #name ra_d dec_d i q u v emaj_s emin_s pa_d

MyLSM = []
if CatChosen == 'SUMSS':
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['Sp'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))
        
elif CatChosen == 'NVSS':
    
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['S1.4'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))

In [5]:

RA = raw_input("Enter RA of target hh mm ss ")
DEC = raw_input("Enter Dec of target dd mm ss ")

Enter RA of target hh mm ss 0 0 0
Enter Dec of target dd mm ss -31 0 0 


In [6]:
MyRadius = raw_input('Enter search radius e.g. 16m, 16d, 16s ')

Enter rsearch radius e.g. 16m, 16d, 16s 30d


In [7]:
def InputCoord(arg1,arg2):
    '''This takes a user input of the RA and Dec and then 
    convert it to RaTarget and DecTarget in deg for use in the VO search.
    Input: arg1(RA), arg2(Dec) NB: They are str in this case.
    Output: RaTarget,DecTarget
    '''
    hh = float(arg1.split(' ')[0])
    mm = float(arg1.split(' ')[1])/60.
    ss = float(arg1.split(' ')[2])/3600.

    dec_dd = float(arg2.split(' ')[0])
    dec_mm = float(arg2.split(' ')[1])/60.
    dec_ss = float(arg2.split(' ')[2])/3600.

    if dec_dd <0:
        dec = dec_dd-dec_mm-dec_ss
    else:
        dec = dec_dd+dec_mm+dec_ss

    RaTarget = (hh+mm+ss)*15.
    DecTarget = dec
    
    return RaTarget,DecTarget

In [8]:
Ra,Dec = InputCoord(RA, DEC)

In [9]:
Dec 

-31.0

In [10]:
# The commonly used Radio catalogues on Vizier is given by:
# first_cat(VIII/92/first14)   
# nvss_cat(VIII/65/nvss)
# SUMSS_cat(VIII/81B/sumss212)


if (Dec > -90.) and (Dec < -30.):
    print("Recommend one use SUMSS survey ")
    catalog_list = Vizier.find_catalogs('VIII/81B/sumss212') 

    result = Vizier.query_region(coord.SkyCoord(ra=Ra, dec=Dec,unit=(u.deg, u.deg),frame='icrs')
                                 ,radius=MyRadius,catalog=catalog_list.keys())

elif (Dec > -30.) and (Dec < 90.):
    print("Recommend to use NVSS or FIRST if Dec >-10 ")
    catalog_list = Vizier.find_catalogs('VIII/65/nvss') 

    result = Vizier.query_region(coord.SkyCoord(ra=Ra, dec=Dec,unit=(u.deg, u.deg),frame='icrs')
                                 ,radius=MyRadius,catalog=catalog_list.keys())
else:
    print("Check your input ")


Recommend one use SUMSS survey 


In [11]:
# Printing catalogues that are accessible
for i in range(len(result)):
    print i,'-',result[i].meta['description']

0 - Observed radio source identifications
1 - Optical and infrared data
2 - Discoverers, Designations, and Positions of True and Possible Planetary Nebulae (1)
3 - IRAS Point Source Catalogue fluxes (1)
4 - Objects rejected as PN (table 3)
5 - X-ray and radio parameters of XMM-Newton source cross-matched radio sources from NVSS, MGPS2 and SUMSS
6 - Observed data 510 extragalactic radio sources
7 - Derived values
8 - Redshift-corrected data
9 - Redshift-corrected derived values
10 - The RC2 catalogue
11 - General list of objects
12 - Observed and Derived parameters (tables 1, 2 and 4 of the publication)
13 - Main catalogue
14 - Sources close to coverage gap
15 - 2QZ catalogue: includes North and South galactic poles, and 6QZ with another 1657 lines.
16 - Additional spectra when more than 2 spectra were taken
17 - dat.The sample of high-redshift galaxies: spectral indeces, redshifts, magnitudes, sizes and flux density for 2442 galaxies.
18 - Quasars (M_v_<-22.25), BL Lac and AGNs (M_v_>-

In [12]:
indx = int(raw_input("Enter catalogue number to use "))

Enter catalogue number to use 69


In [13]:
MyCat = result[result.keys()[indx]]
print "Catalogue chosen is: ", result[indx].meta['description']
MyCat[:5]

Catalogue chosen is:  The SUMSS Catalog, Version 2.1r (Feb-16-2012)


RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Sp,e_Sp,St,e_St,MajAxis,MinAxis,PA,Mosaic,Nm
"""h:m:s""",arcs,"""d:m:s""",arcs,mJy,mJy,mJy,mJy,arcs,arcs,deg,,
str11,float32,str12,float32,float32,float32,float64,float32,float32,float32,float32,str8,uint8
00 00 00.00,3.6,-31 09 53.10,4.7,13.1,1.4,13.2,1.4,84.9,58.4,162.3,J0000M32,1
00 00 00.03,3.1,-50 28 08.50,3.6,7.4,0.8,7.4,0.8,57.1,46.6,163.8,J0000M52,1
00 00 00.10,2.1,-45 57 21.50,2.5,16.7,1.0,20.9,1.3,70.0,50.1,17.1,J0000M48,2
00 00 00.26,1.8,-47 37 57.70,2.0,28.1,1.3,28.1,1.3,60.6,48.7,158.9,J0000M48,1
00 00 01.72,3.3,-59 36 16.30,3.5,11.1,1.2,11.1,1.2,52.0,47.0,151.8,J0000M60,1


In [17]:
len(MyCat)

50

In [18]:
if "SUMSS" in result[indx].meta['description']:
    CatChosen = 'SUMSS'
elif "NVSS" in result[indx].meta['description']:
    CatChosen = 'NVSS'
else:
    print("Non standard cat chosen may break code")

In [19]:
# LSM should be of the following format:
# #name ra_d dec_d i q u v emaj_s emin_s pa_d

MyLSM = []
if CatChosen == 'SUMSS':
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['Sp'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))
        
elif CatChosen == 'NVSS':
    
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['S1.4'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))

In [20]:
MyLSMHeader = '#format:name ra_d dec_d i q u v emaj_s emin_s pa_d'

In [21]:
np.savetxt('test_LSM.lsm', np.array(MyLSM),header=MyLSMHeader,comments='',
           fmt= '%i %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e')

In [23]:
!ls -lah

total 45G
drwxr-xr-x  7 2020 1000 4.0K Nov 12 11:32 .
drwxr-xr-x 18 2020 1000 4.0K Nov  2 09:34 ..
drwxr-xr-x  8 2020 1000  257 Nov 12 11:20 .git
-rw-r--r--  1 2020 1000  163 Nov 12 11:19 .gitignore
drwxr-xr-x  2 root root  172 Nov 12 11:26 .ipynb_checkpoints
-rw-r--r--  1 2020 1000 1.6K Nov 11 15:03 README.md
-rw-r--r--  1 2020 1000 8.2K Nov 12 11:16 RFIsim.py
-rw-r--r--  1 2020 1000  20K Nov 12 11:30 VO_example_for_Sky_model.ipynb
-rw-r--r--  1 2020 1000  49K Nov 12 11:32 VO_example_for_Sky_model_w_NVSS_n_SUMSS.ipynb
-rw-r--r--  1 2020 root 3.4M Nov 11 18:25 montblanc.log
drwxr-xr-x  3 2020 1000  333 Nov 12 11:20 notebooks
drwxr-xr-x  2 2020 1000   10 Nov  8 16:01 output_data
-rw-r--r--  1 root root  45G Nov 11 18:25 ra=0.0_dec=0.0_int_secs=8_track-time=0.1hrs_nants=64_nchan=4096.h5
-rw-r--r--  1 root root 5.6K Nov 12 11:32 test_LSM.lsm
drwxr-xr-x  6 2020 1000  197 Nov 12 11:16 utils


In [19]:
# Copy the sky model to the path you want

In [24]:
import Tigger

In [25]:
lsm = Tigger.load('test_LSM.lsm')

Loading test_LSM.lsm: ASCII table


In [27]:
len(lsm.sources)

50

## SUMSS Catalogue 

www.astrop.physics.usyd.edu.au/sumsscat/sumsscat.Mar-11-2008.gz

## NVSS Catalogue

ftp://nvss.cv.nrao.edu/pub/nvss/CATALOG/NVSSCatalog.text.gz

In [31]:
nant = 64
A1, A2 = np.empty(nant*(nant-1)/2, dtype=np.int32), np.empty(nant*(nant-1)/2, dtype=np.int32)
k = 0
for i in range(nant):
    for j in range(i+1,nant):
        A1[k] = i
        A2[k] = j
        k += 1

corr_products = np.array(['m{:03d}m{:03d}'.format(A1[i], A2[i]) for i in range(len(A1))])

corr_ex = ['m002m004', 'm002m063']

df = pd.DataFrame(data=np.arange(2016), index=corr_products).T

df[corr_ex].values